# 导入库函数

In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Parameter
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# evaluating indicator

In [2]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    ap = average_precision_score(true_edges, predict_edges)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    return ap, auc
    #predict_edges[predict_edges>gama] = 1
    #predict_edges[predict_edges<=gama] = 0
    
    #print("ACC： ",accuracy_score(true_edges, predict_edges))
    #print("AP： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("RECALL： ",recall_score(true_edges, predict_edges, average='macro'))
    #print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))

# model

In [3]:
"""  Given the cluster centroids, update the node representation  """
class reconstruction_graph(nn.Module):
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, CC):
        # Calculate the attention score (in nodes) for each node to each cluster
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # Distance from the node to the center of mass
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power    
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
        prod = torch.mm(soft_assignments, soft_assignments.t())       #  numerator
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # Calculate the distance between nodes: smaller values favor edge formation
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, CC, edge_train, edge_test):
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        ap, auc = predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach(), ap, auc
    
"""  Type-centroid update aimed at maximizing the prediction effect   """
class ClusteringLayer(nn.Module):
    """  Given the node representation, update the cluster centroids  """
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, NE):
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2) 
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power   
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() 
        
        prod = torch.mm(soft_assignments, soft_assignments.t())
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(NE, CC, g, edge_train, edge_test, num_epochs=5):        
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=LR, momentum=0.9)
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(5):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    ap, auc = predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach(), ap, auc

In [4]:
def get_graph(nodes_number):
    edges = pd.read_csv('datasets/polbooks.csv', header=None).iloc[1:,]
    edges = edges.astype('int')
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[0]):
        graph_np[edges.iloc[i, 0], edges.iloc[i, 1]]=1
        graph_np[edges.iloc[i, 1], edges.iloc[i, 0]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # Only the upper-left corner matrix is taken
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # Only the upper-left corner matrix is taken
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*6,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [5]:
nodes_number = 105

In [6]:
embedding_dim = 8
n_clusters = 12
alpha = 1
beta = 4.5
DEC = 4.5
LR = 0.4

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask = get_graph(nodes_number)
    ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)       
    raw_nodes_embedding = ini_embedding.weight.detach()        # Initial node embedding

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids
    
    ap_first__ap = 0
    ap_first_auc = 0
    auc_first__ap = 0
    auc_first_auc = 0
    for module_epoch in range(300):
        print("######################### Module cycle ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
        raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
        if nodes_ap > ap_first__ap:
            ap_first__ap = nodes_ap
            ap_first_auc = nodes_auc
        if nodes_auc > auc_first_auc:
            auc_first_auc = nodes_auc
            auc_first__ap = nodes_ap
        raw_cluster_centers, cluster_ap, cluster_auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)
        if cluster_ap > ap_first__ap:
            ap_first__ap = cluster_ap
            ap_first_auc = cluster_auc
        if cluster_auc > auc_first_auc:
            auc_first_auc = cluster_auc
            auc_first__ap = cluster_ap
    all_ap_first_ap.append(ap_first__ap)
    all_ap_first_auc.append(ap_first_auc)
    all_auc_first_ap.append(auc_first__ap)
    all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
AP：  0.6592125001722536
AUC SCORE:  0.6009876543209877
AP：  0.6251905695417307
AUC SCORE:  0.5604938271604938
######################### Module cycle ： 1 ##########################
AP：  0.7333301600486036
AUC SCORE:  0.688395061728395
AP：  0.7594054921545459
AUC SCORE:  0.7264197530864198
######################### Module cycle ： 2 ##########################
AP：  0.7898573438508842
AUC SCORE:  0.7560493827160494
AP：  0.8123058052329964
AUC SCORE:  0.7871604938271605
######################### Module cycle ： 3 ##########################
AP：  0.847529263065676
AUC SCORE:  0.8271604938271604
AP：  0.8615869097959532
AUC SCORE:  0.8469135802469135
######################### Module cycle ： 4 ##########################
AP：  0.8668191009288039
AUC SCORE:  0.854320987654321
AP：  0.8702079213326455
AUC SCORE:  0.8587654320987654
######################### Module cycle ： 5 ##########################
AP：  0.8795515554621096
AUC SCORE

AUC SCORE:  0.9165432098765433
######################### Module cycle ： 46 ##########################
AP：  0.9049816636338306
AUC SCORE:  0.9165432098765433
AP：  0.906242154095773
AUC SCORE:  0.9175308641975308
######################### Module cycle ： 47 ##########################
AP：  0.9057205798899883
AUC SCORE:  0.917037037037037
AP：  0.906242154095773
AUC SCORE:  0.9175308641975308
######################### Module cycle ： 48 ##########################
AP：  0.906242154095773
AUC SCORE:  0.9175308641975308
AP：  0.906242154095773
AUC SCORE:  0.9175308641975308
######################### Module cycle ： 49 ##########################
AP：  0.9064567673042405
AUC SCORE:  0.9180246913580247
AP：  0.9064567673042405
AUC SCORE:  0.9180246913580247
######################### Module cycle ： 50 ##########################
AP：  0.9064567673042405
AUC SCORE:  0.9180246913580247
AP：  0.9058778690782835
AUC SCORE:  0.9175308641975308
######################### Module cycle ： 51 #########################

AUC SCORE:  0.9219753086419753
######################### Module cycle ： 95 ##########################
AP：  0.911619399502719
AUC SCORE:  0.9214814814814815
AP：  0.9121292210651721
AUC SCORE:  0.9219753086419753
######################### Module cycle ： 96 ##########################
AP：  0.911619399502719
AUC SCORE:  0.9214814814814815
AP：  0.9121292210651721
AUC SCORE:  0.9219753086419753
######################### Module cycle ： 97 ##########################
AP：  0.911619399502719
AUC SCORE:  0.9214814814814815
AP：  0.9121292210651721
AUC SCORE:  0.9219753086419753
######################### Module cycle ： 98 ##########################
AP：  0.911619399502719
AUC SCORE:  0.9214814814814815
AP：  0.9121292210651721
AUC SCORE:  0.9219753086419753
######################### Module cycle ： 99 ##########################
AP：  0.911619399502719
AUC SCORE:  0.9214814814814815
AP：  0.9121292210651721
AUC SCORE:  0.9219753086419753
######################### Module cycle ： 100 ########################

AUC SCORE:  0.9264197530864198
AP：  0.9161029699464645
AUC SCORE:  0.9264197530864198
######################### Module cycle ： 141 ##########################
AP：  0.9163100092839386
AUC SCORE:  0.9269135802469136
AP：  0.9161029699464645
AUC SCORE:  0.9264197530864198
######################### Module cycle ： 142 ##########################
AP：  0.9163100092839386
AUC SCORE:  0.9269135802469136
AP：  0.9163100092839386
AUC SCORE:  0.9269135802469136
######################### Module cycle ： 143 ##########################
AP：  0.9163100092839386
AUC SCORE:  0.9269135802469136
AP：  0.9163100092839386
AUC SCORE:  0.9269135802469136
######################### Module cycle ： 144 ##########################
AP：  0.9163100092839386
AUC SCORE:  0.9269135802469136
AP：  0.9163100092839386
AUC SCORE:  0.9269135802469136
######################### Module cycle ： 145 ##########################
AP：  0.9163100092839386
AUC SCORE:  0.9269135802469136
AP：  0.9161313408195558
AUC SCORE:  0.9269135802469136
####

AUC SCORE:  0.928395061728395
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
######################### Module cycle ： 188 ##########################
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
######################### Module cycle ： 189 ##########################
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
######################### Module cycle ： 190 ##########################
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
######################### Module cycle ： 191 ##########################
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
######################### Module cycle ： 192 ##########################
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
AP：  0.9169589897294451
AUC SCORE:  0.928395061728395
################

...

AP：  0.931006254689624
AUC SCORE:  0.9254320987654321
AP：  0.931006254689624
AUC SCORE:  0.9254320987654321
######################### Module cycle ： 64 ##########################
AP：  0.931006254689624
AUC SCORE:  0.9254320987654321
AP：  0.931006254689624
AUC SCORE:  0.9254320987654321
######################### Module cycle ： 65 ##########################
AP：  0.931006254689624
AUC SCORE:  0.9254320987654321
AP：  0.931006254689624
AUC SCORE:  0.9254320987654321
######################### Module cycle ： 66 ##########################
AP：  0.931006254689624
AUC SCORE:  0.9254320987654321
AP：  0.931006254689624
AUC SCORE:  0.9254320987654321
######################### Module cycle ： 67 ##########################
AP：  0.931006254689624
AUC SCORE:  0.9254320987654321
AP：  0.9313805651703174
AUC SCORE:  0.9259259259259259
######################### Module cycle ： 68 ##########################
AP：  0.9313805651703174
AUC SCORE:  0.9259259259259259
AP：  0.9313805651703174
AUC SCORE:  0.92592592592

AP：  0.932858390892517
AUC SCORE:  0.9274074074074073
AP：  0.9326571836490564
AUC SCORE:  0.9269135802469135
######################### Module cycle ： 114 ##########################
AP：  0.932858390892517
AUC SCORE:  0.9274074074074073
AP：  0.9326571836490564
AUC SCORE:  0.9269135802469135
######################### Module cycle ： 115 ##########################
AP：  0.9326080637604695
AUC SCORE:  0.9269135802469136
AP：  0.932858390892517
AUC SCORE:  0.9274074074074073
######################### Module cycle ： 116 ##########################
AP：  0.9326080637604695
AUC SCORE:  0.9269135802469136
AP：  0.9326080637604695
AUC SCORE:  0.9269135802469136
######################### Module cycle ： 117 ##########################
AP：  0.9326080637604695
AUC SCORE:  0.9269135802469136
AP：  0.9326080637604695
AUC SCORE:  0.9269135802469136
######################### Module cycle ： 118 ##########################
AP：  0.9326080637604695
AUC SCORE:  0.9269135802469136
AP：  0.9326080637604695
AUC SCORE:  0.

AP：  0.9318022031434506
AUC SCORE:  0.9264197530864198
AP：  0.931509805482632
AUC SCORE:  0.9259259259259258
######################### Module cycle ： 164 ##########################
AP：  0.931509805482632
AUC SCORE:  0.9259259259259258
AP：  0.931509805482632
AUC SCORE:  0.9259259259259258
######################### Module cycle ： 165 ##########################
AP：  0.931509805482632
AUC SCORE:  0.9259259259259258
AP：  0.931509805482632
AUC SCORE:  0.9259259259259258
######################### Module cycle ： 166 ##########################
AP：  0.931509805482632
AUC SCORE:  0.9259259259259258
AP：  0.9318635469792306
AUC SCORE:  0.9264197530864198
######################### Module cycle ： 167 ##########################
AP：  0.9318635469792306
AUC SCORE:  0.9264197530864198
AP：  0.9318635469792306
AUC SCORE:  0.9264197530864198
######################### Module cycle ： 168 ##########################
AP：  0.9318635469792306
AUC SCORE:  0.9264197530864198
AP：  0.9318635469792306
AUC SCORE:  0.926

AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
######################### Module cycle ： 215 ##########################
AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
######################### Module cycle ： 216 ##########################
AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
######################### Module cycle ： 217 ##########################
AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
######################### Module cycle ： 218 ##########################
AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
######################### Module cycle ： 219 ##########################
AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
AP：  0.9315843080676128
AUC SCORE:  0.9259259259259259
######################### Module cy

AUC SCORE:  0.9254320987654321
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
######################### Module cycle ： 264 ##########################
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
######################### Module cycle ： 265 ##########################
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
######################### Module cycle ： 266 ##########################
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
######################### Module cycle ： 267 ##########################
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
######################### Module cycle ： 268 ##########################
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
AP：  0.9312014722259414
AUC SCORE:  0.9254320987654321
####

In [8]:
print("################################  SUM UP #####################################")
print("AP FIRST AP MEAN: ", np.array(all_ap_first_ap).mean())
print("AP FIRST AP STD: ", np.array(all_ap_first_ap).std())

print("AP FIRST AUC MEAN: ", np.array(all_ap_first_auc).mean())
print("AP FIRST AUC STD: ", np.array(all_ap_first_auc).std())

print("AUC FIRST AP MEAN: ", np.array(all_auc_first_ap).mean())
print("AUC FIRST AP STD: ", np.array(all_auc_first_ap).std())

print("AUC FIRST AUC MEAN: ", np.array(all_auc_first_auc).mean())
print("AUC FIRST AUC STD: ", np.array(all_auc_first_auc).std())

################################  SUM UP #####################################
AP FIRST AP MEAN:  0.9145780959664485
AP FIRST AP STD:  0.033316071631511825
AP FIRST AUC MEAN:  0.9113086419753088
AP FIRST AUC STD:  0.0360231259020665
AUC FIRST AP MEAN:  0.9108423895361127
AUC FIRST AP STD:  0.03692499327522389
AUC FIRST AUC MEAN:  0.9133827160493828
AUC FIRST AUC STD:  0.03419852028697478
